# Downloading GUNW products using ariaDownload.py

**Author**: Brett A. Buzzanga, David Bekaert - Jet Propulsion Laboratory

This notebook demonstrates how to use the productAPI.py command line tool to download Sentinel 1 ARIA Geocoded UNWrapped interferogram (**GUNW**) products.  A detailed overview of the ARIA GUNW product with respect to processing, formatting, sampling, and data layers can be found on the [ARIA website](https://aria.jpl.nasa.gov/node/97).

The **`ariaDownload.py`** program wraps around the NASA's ASF DAAC API and [Bulk Download Service](https://bulk-download.asf.alaska.edu/help). The ASF Bulk Download Service handles most of the heavy lifting of the data-download and will conveniently skip previously downloaded files, and re-download partially downloaded files.  
In this notebook, we will demonstrate **`ariaDownload.py`** functionality along track 4, which intersects the U.S. East Coast in southeastern Virginia.


<div class="alert alert-warning">
<b>Potential download failure:</b> 
GUNW products are hosted at the NASA ASF DAAC. Downloading them requires a NASA Earthdata URS user login and requires users to add "GRFN Door (PROD)" to their URS approved applications

<b>Login Credentials:</b>
Save your user-name and password to a `.netrc` file in your `$HOME` directory, or pass the combination explicitly using `ariaDownload.py --user <user> --pass <pass>`.


To create a .netrc file, pass your earthdata credentials as so:
```
echo "machine urs.earthdata.nasa.gov login <user> password <pass>" > ~/.netrc chmod 600 ~/.netrc
```
</div>

## Overview of the ariaDownload.py program

Running **`ariaDownload.py`** with no options, or with **`-h`**, will show the parameters options as well as some examples. At minimum, users need to specify a spatial constraint either as a track number or bounding box (can be a shapefile).

Let us explore what some of the other options are:

In [1]:
!ariaDownload.py

ARIA-tools Version: 1.1.0
usage: ariaDownload.py [-h] [-o OUTPUT] [-t TRACK] [-b BBOX] [-w WD]
                       [-s START] [-e END] [-u USER] [-p PASSW] [-l DAYSLT]
                       [-m DAYSGT] [-nt NUM_THREADS] [-i IFG] [-d FLIGHTDIR]
                       [--version VERSION] [-v]

Command line interface to download GUNW products from the ASF DAAC. GUNW products are hosted at the NASA ASF DAAC.

optional arguments:
  -h, --help            show this help message and exit
  -o OUTPUT, --output OUTPUT
                        Output type, default is "Download". "Download",
                        "Count", and "Url""Kmz" are currently supported. Use
                        "Url" for ingestion to aria*.py
  -t TRACK, --track TRACK
                        track to download; single number (including leading
                        zeros) or comma separated
  -b BBOX, --bbox BBOX  Lat/Lon Bounding SNWE, or GDAL-readable file
                        containing POLYGON geometry.
  -

## 1. Example of ariaDownload output options

The user has the option to **download**, **count**, and generate a **kmz** corresponding to return of their search criteria. Below we provide example for each of these over Track 4.

### Count the number of products

To get a count of the number of products, without downloading data, provide the **`--output`** option with the **`count`** argument. To get information on the exact product filenames also include the verbose option **`-v`**.

In [2]:
!ariaDownload.py --track 4 --output count

ARIA-tools Version: 1.1.0

Found -- 3907 -- products


### Generate list of virtual products from ASF S3 bucket (BETA)

To generate a textfile containing a list of product URLs from the ASF S3 bucket, without downloading data, provide the **`--output`** option with the **`url`** argument. To get information on the exact product filenames also include the verbose option **`-v`**. Extracting layers virtually by leveraging this list of URLs is currently only supported by systems with the following packages: Linux kernel >=4.3 and libnetcdf >=4.5 

In [3]:
!ariaDownload.py --track 4 --output url

ARIA-tools Version: 1.1.0
Wrote -- 3907 -- product urls to: /Users/buzzanga/Software_InSAR/ARIA-tools-docs_git/JupyterDocs/ariaDownload/products/track4_0.txt


We can now have a look at the generated textfile, which contains the URLs of all standard products over the specified track. As there are a lot of products we will only have a look at the first 10.

In [4]:
!head -n 10 products/download_products_4track_0.txt

head: products/download_products_4track_0.txt: No such file or directory


### Generate KMZ of products

To generate a Google Earth kmz, without downloading data, provide the **`--output`** option with the **`kmz`** argument. In case you also want to get information on the exact product filenames you can enbale the verbose option by adding **`-v`**. The result is shown in **Fig. 1**.

In [5]:
!ariaDownload.py --track 4 --output kmz

ARIA-tools Version: 1.1.0
Traceback (most recent call last):
  File "/Users/buzzanga/Miniconda3/envs/ARIA/bin/ariaDownload.py", line 7, in <module>
    exec(compile(f.read(), __file__, 'exec'))
  File "/Users/buzzanga/Software_InSAR/ARIA-tools_git/tools/bin/ariaDownload.py", line 293, in <module>
    Downloader(inps)()
  File "/Users/buzzanga/Software_InSAR/ARIA-tools_git/tools/bin/ariaDownload.py", line 239, in __call__
    dst    = self._fmt_dst()
AttributeError: 'Downloader' object has no attribute '_fmt_dst'


<img src="./support_docs/track_004.png" alt="track" width="700">
<center><b>Fig. 1</b> Image of products available along Track 4. </center>

### Download products

To download the products, simply omit the **`--output`** option or specify it with the **`download`** argument. Examples for downloading all data on track 4 include:

`ariaDownload.py -t 004`

`ariaDownload.py -t 004 -o download`

If user/pass are not explicitly passed, the wrapper will look for a `.netrc` in the `$HOME` directory. Alternatively, you can pass them directly as `ariaDownload.py --user <USER> --pass <PASS>`. Otherwise, you will be prompted to enter them.

By default, the products will be downloaded into the *./products* folder. You can specify a different location using the **`-w`** option, such as:

`ariaDownload.py -t 004 -w /insarHome`

<div class="alert alert-warning">
<b>Potential download failure:</b>
GUNW products are hosted at the NASA ASF DAAC. Downloading them requires a NASA Earthdata URS user login and requires users to add "GRFN Door (PROD)" to their URS approved applications
</div>

## 2. Spatial-temporal subsetting

The **`ariaDownload.py`** program has a number of options availble for subsetting the search of products in the spatial and temporal domain, including **bounding box**, **start/end date**, **aquisition geometry**, **temporal baseline**, or **interferogram** options. Below, using the count option for brevity, we demonstrate each of them.

### Subsetting by Bounding Box

Bounding box can either be specified as **SNWE** coordinates (a string with quotation) or by providing a **shapefile** (GeoJSON or ESRI shapefiles) to the **`--bbox`** option. 

An example based on subsetting using **SNWE** coordinates and the corresponding kmz is shown in **Fig. 2**:

In [6]:
!ariaDownload.py --track 4 --bbox "36.75 37.225 -76.655 -75.928" -o count

ARIA-tools Version: 1.1.0

Found -- 551 -- products


<img src="./support_docs/crop_hr.png" alt="crop" width="700">

<center><b>Fig. 2</b> Image of products available along within bounding box 36.75S, 37.225N, -76.655W, -75.928E</center>

An example based on subsetting using a shape file:

In [7]:
!ariaDownload.py --track 4 --bbox './support_docs/HR_North.shp' -o count

ARIA-tools Version: 1.1.0

Found -- 1092 -- products


<img src="./support_docs/crop_shp.png" alt="crop" width="700">

<center><b>Fig. 3</b> Image of products available along within shapefile (blue shading). </center>

### Subsetting by start/end date

Subsetting in time can be done by specifying a range within which products need to fall. This is controlled using the **`--start YYYYMMDD` ** and **`--end YYYYMMDD`** options. By default the complete observational record is considered.

In [8]:
!ariaDownload.py --track 4 --bbox "36.75 37.225 -76.655 -75.928" -o count --start 20190101 

ARIA-tools Version: 1.1.0

Found -- 195 -- products


In [9]:
!ariaDownload.py --track 4 --bbox "36.75 37.225 -76.655 -75.928" -o count -s 20190101  --end 20190401 --verbose

ARIA-tools Version: 1.1.0

Found -- 12 -- products


Note the use of the **`--verbose`** option to show the products found.

### Subsetting based on product temporal baseline

The **`--daysless`** and **`--daysmore`** options allow us to subset based on the temporal baseline, i.e. the time length (in days) between the reference and secondary SAR acquisitions of the interferogram. Note that `--daysless 12` and `--daysmore 12` specifies only pairs with temporal baselines smaller and larger than 12 (so not exactly separated by 12) are downloaded, respectively.

To select only interferograms products with temporal baselines shorter than 13 days:

In [10]:
!ariaDownload.py --track 4 --bbox "36.75 37.225 -76.655 -75.928" -o count --daysless 13 -v

ARIA-tools Version: 1.1.0

Found -- 118 -- products


Select only pairs with temporal baselines, greater than a year:

In [11]:
!ariaDownload.py --track 4 --bbox "36.75 37.225 -76.655 -75.928" -o count --daysmore 364

ARIA-tools Version: 1.1.0

Found -- 177 -- products


Select only pairs with moderate temporal baselines between two and six months:

In [12]:
!ariaDownload.py --track 4  --bbox "36.75 37.225 -76.655 -75.928"  -o count -m 59 -l 181 -v 

ARIA-tools Version: 1.1.0

Found -- 10 -- products


### Subsetting based on interferogram pair

To find a specific interferogram combination one can use the **`--ifg YYYYMMDD_YYYYMMDD`** option.

In [13]:
!ariaDownload.py -b "36.75 37.225 -76.655 -75.928"  -o count --ifg "20161018_20160702" -v 

ARIA-tools Version: 1.1.0

Found -- 1 -- products


### Testing your URS credentials set-up

We will now verify if all is set for you to downlaod products, per instructions at the top of the notebook.

In [14]:
!ariaDownload.py -b "36.75 37.225 -76.655 -75.928"   --ifg "20161018_20160702" -v 

ARIA-tools Version: 1.1.0
Wrote -- 1 -- products to: /Users/buzzanga/Software_InSAR/ARIA-tools-docs_git/JupyterDocs/ariaDownload/products
